In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# fill missing data

In [172]:
df = pd.read_csv('london_weather.csv')
df = df.fillna(1)


In [173]:
x = df.drop(columns=['max_temp'], axis=1)  
y = df['max_temp']  

In [174]:
x = np.array(x)
y = np.array(y)

In [175]:
from sklearn.model_selection import train_test_split

x, x_test_main, y, y_test_main = train_test_split(x, y, test_size=0.1, random_state=42)
# y_train = y_train.astype(int)
# y_test = y_test.astype(int)
y_test_main.shape

(1535,)

In [176]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
x_test_main = scaler.fit_transform(x_test_main)
x_train[0]

array([ 0.82029225, -0.23225363,  0.03921345,  0.15777323,  0.01391402,
        0.00360213, -0.00134418, -0.00594078])

# pca

In [177]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=8)

# x = pca.fit_transform(X=x)
# x_test_main = pca.transform(X=x_test_main)

# define model 1

In [178]:
# def get_reg(meta, hidden_layer_sizes, dropout, active='relu'):
#     n_features_in_ = meta["n_features_in_"]
#     model = keras.models.Sequential()
#     model.add(keras.layers.Input(shape=(n_features_in_,)))
#     for hidden_layer_size in hidden_layer_sizes:
#         model.add(keras.layers.Dense(hidden_layer_size, activation=active))
#         model.add(keras.layers.Dropout(dropout))
#     model.add(keras.layers.Dense(1))
#     return model

In [179]:
# from scikeras.wrappers import KerasRegressor


# reg = KerasRegressor(
#     model=get_reg,
#     loss="mse",
#     metrics=[keras.metrics.R2Score],
#     hidden_layer_sizes=(100,),
#     dropout=0.5,
# )
# reg.fit(x_train, y_train)

In [180]:
# from sklearn.model_selection import RandomizedSearchCV

# param_grid = {
#     'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],
#     'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
# }
# rs = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, cv=3, n_iter=20)

# rs.fit(x_train, y_train)


# define 2

In [181]:
# from sklearn.model_selection import RandomizedSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier

# # Define a function to create your Keras model
# def create_model(activation='relu', hidden_layer_size=32, dropout_rate=0.2):
#     # ... (Your model architecture)

#     return model

# # Create a KerasClassifier instance
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define the hyperparameter search space
# param_dist = {
#     'activation': ['relu', 'tanh', 'sigmoid'],
#     'hidden_layer_size': [16, 32, 64, 128],
#     'dropout_rate': [0.2, 0.3, 0.4, 0.5]
# }

# # Create a RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3)

# # Fit the random search to your data
# random_search.fit(X_train, y_train)

# # Get the best parameters and model
# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# define 3

In [182]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kf = KFold(n_splits=6)

In [183]:
def model_creator():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(9,)))
    model.add(keras.layers.Dense(units=32, activation='relu'))
    model.add(keras.layers.Dense(units=32, activation='relu'))
    model.add(keras.layers.Dense(units=1, activation='linear'))
    model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
    return model




In [184]:
mse_list = []
for train, validation in kf.split(x, y):

    # x_train = x_train_main[train]
    # y_train = y_train_main[train]

    # x_test = x_train_main[validation]
    # y_test = x_train_main[validation]

    # x_train, x_test = x[train], x[validation]
    # y_train, y_test = y[train], y[validation]

    x_train = x[train]
    x_test = x[validation]
    y_train = y[train]
    y_test = y[validation]
    
    model = model_creator()
    model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

    y_pred = model.predict(x_test)

    mse_list.append(mean_squared_error(y_test, y_pred))

mse_list





# 6.304961665296364 => mean_squared_error with scaler and without pca




Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0013 - loss: 231.5578 - val_accuracy: 0.0000e+00 - val_loss: 25.1570
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0013 - loss: 23.0328 - val_accuracy: 0.0000e+00 - val_loss: 14.7933
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 7.1042e-04 - loss: 12.7208 - val_accuracy: 0.0000e+00 - val_loss: 8.4603
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 9.0482e-04 - loss: 8.0451 - val_accuracy: 0.0000e+00 - val_loss: 7.5558
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0015 - loss: 7.4140 - val_accuracy: 0.0000e+00 - val_loss: 7.3856
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0015 - loss: 7.3882 - val_accuracy: 0.0000e+00 - val_loss: 7.2946
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 8.4181e-04 - loss: 7.1645 - val_accuracy: 0.0000e+00 - val_loss: 7.3089
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accu

[6.397143984992108,
 6.5783066080602115,
 6.7221922268464125,
 6.4200810615036925,
 6.3737831938365055,
 6.180065590768854]

In [185]:
np.mean(mse_list)


6.4452621110012975

In [186]:
y_pred_main = model.predict(x_test_main)
mse_main = mean_squared_error(y_test_main, y_pred_main)
mse_main

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


6.934512681786532

# define 4

In [187]:
# model = keras.Sequential()
# model.add(keras.layers.Input(shape=(9,)))
# model.add(keras.layers.Dense(units=32, activation='relu'))
# model.add(keras.layers.Dense(units=32, activation='sigmoid'))
# model.add(keras.layers.Dense(units=1))
# model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
# # model.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.mean_squared_error, metrics=['accuracy'])
# model.fit(x, y, epochs=20, batch_size=64, validation_data=(x_test, y_test))
# y_pred = model.predict(x_test_main)
# mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
# mse
# # 12.187250703339979 => mean_squared_error with scaler and pca
# # 10.015396159320842 => mean_squared_error with scaler and without pca
# # 42.39169921336769 => mean_squared_error without scaler and without pca

# in simple model just do MinMaxScaler got the best result